In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ansys.mapdl.core import launch_mapdl

In [2]:
mapdl = launch_mapdl()

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...


In [3]:
mapdl.clear("NOSTART")
mapdl.prep7()
mapdl.units("SI")  # SI-International system (m, kg, s, K).

mapdl.et(1, "SOLID186")

# Define a material (nominal steel in SI)
mapdl.mp("EX", 1, 210e9)  # Elastic moduli in Pa (kg/(m*s**2))
mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
mapdl.mp("NUXY", 1, 0.3)  # Poisson's Ratio
mapdl.emodif("ALL", "MAT", 1)

*** NOTE ***                            CP =       0.266   TIME= 23:17:55
 No elements defined.

In [4]:
# Geometry
    
# Create a rectangular area with the hole in the middle
center_hole_dia = 0.045 # Center hole diameter = 45 mm
sat_holes_dia = 0.03 # Satellite holes diameter = 30 mm
sat_holes_loc = 0.05 # Left satellite holes location at x = 50 mm along y = 50 mm axis

# Create the rectangle
rect_anum = mapdl.blc4(width = 0.2, height = 0.1) # 200 mm x 100 mm

# Create a circle in the middle of the rectangle
circ_anum = mapdl.cyl4(0.1, 0.05, center_hole_dia/2)
    
# Create a satellite circle at locations x = 50 mm along y = 50 mm axis
sat_circ_anum = mapdl.cyl4(sat_holes_loc ,0.05, sat_holes_dia/2)
sat_circ_anum2 = mapdl.cyl4(0.2-sat_holes_loc ,0.05, sat_holes_dia/2) # (200 mm - left hole location) = right hole location

# Cut the circle out of the rectangle.
area1= mapdl.asba(rect_anum, circ_anum)
area2 = mapdl.asba(area1, sat_circ_anum)
plate_with_hole_area = mapdl.asba(area2, sat_circ_anum2)

plate_with_hole_anum=mapdl.vext(plate_with_hole_area, dz=0.003) # make volume from area

In [5]:
mapdl.lplot(cpos="xy", line_width=3, font_size=26, color_lines=True, background="w")

Widget(value="<iframe src='http://localhost:55911/index.html?ui=P_0x23439aa0b80_0&reconnect=auto' style='width…

In [6]:
mapdl.lesize("ALL", 0.005, layer1=1)

mapdl.mshape(1, "3D")
mapdl.mshkey(0)

mapdl.vmesh('all')
mapdl.eplot()

Widget(value="<iframe src='http://localhost:55911/index.html?ui=P_0x2343b1ea0d0_1&reconnect=auto' style='width…

In [7]:
mapdl.mesh

ANSYS Mesh
  Number of Nodes:              10554
  Number of Elements:           5054
  Number of Element Types:      1
  Number of Node Components:    0
  Number of Element Components: 0

In [8]:
# Boundary Conditions
mapdl.nsel('s','loc','x',0)
mapdl.d('all','all',0)

mapdl.nsel('S','LOC','X', 0.2)
mapdl.nsel("R", "LOC", "Y", 0.1/2)
single_node = mapdl.mesh.nnum[0]
mapdl.nsel("S", "NODE", vmin=single_node, vmax=single_node)
mapdl.f("ALL", "FZ", -100)

mapdl.allsel()
mapdl.finish()

***** ROUTINE COMPLETED *****  CP =         0.969

In [9]:
# Solve the Static Problem

mapdl.run("/SOLU")
mapdl.antype("STATIC")
output=mapdl.solve()
print(output)

*****  MAPDL SOLVE    COMMAND  *****

 *** NOTE ***                            CP =       1.047   TIME= 23:17:57
 There is no title defined for this analysis.                            

 *** SELECTION OF ELEMENT TECHNOLOGIES FOR APPLICABLE ELEMENTS ***
                ---GIVE SUGGESTIONS ONLY---

 ELEMENT TYPE         1 IS SOLID186. KEYOPT(2) IS ALREADY SET AS SUGGESTED.



 *** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   23:17:57  MAR 27, 2023 CP=      1.078

                                                                               



                       S O L U T I O N   O P T I O N S

   PROBLEM DIMENSIONALITY. . . . . . . . . . . . .3-D                  
   DEGREES OF FREEDOM. . . . . . UX   UY   UZ  
   ANALYSIS TYPE . . . . . . . . . . . . . . . . .STATIC (STEADY-STATE)
   GLOBALLY ASSEMBLED MATRIX . . . . . . . . . . .SYMMETRIC  

 *** NOTE ***          

In [10]:
# Post-Processing

# Grab the stress from the result
result = mapdl.result
nnum, stress = result.principal_nodal_stress(0)

result.plot_principal_nodal_stress(0,"SEQV",lighting=False,cpos="xy",background="w",text_color="k",add_text=False,)

Widget(value="<iframe src='http://localhost:55911/index.html?ui=P_0x2343db86e20_2&reconnect=auto' style='width…

In [11]:
stress_left_hole = mapdl.get('sigeqv', 'node','node(0.05,0.05+0.030/2,0)','s','eqv')

In [12]:
stress_center_hole = mapdl.get('sigeqv', 'node','node(0.1,0.05+0.045/2,0)','s','eqv')

In [13]:
nominal_stress_left = 6*(100)*(0.15)/((0.1-0.03)*(0.003**2))
nominal_stress_center = 6*(100)*(0.1)/((0.1-0.045)*(0.003**2))


In [14]:
KT_left = stress_left_hole / nominal_stress_left
KT_center = stress_center_hole / nominal_stress_center

In [15]:
print(f'Stress concentration factor (left hole)= {KT_left:.4f}')
print(f'Stress concentration factor (center hole))= {KT_center:.4f}')

Stress concentration factor (left hole)= 1.3355
Stress concentration factor (center hole))= 1.2317


In [16]:
 #Run Batch using FUNCTION

In [17]:
def compute_stress_con(sat_holes_dia,sat_holes_loc):
    
    mapdl.clear("NOSTART")
    mapdl.prep7()
    mapdl.units("SI")  # SI-International system (m, kg, s, K).

    mapdl.et(1, "SOLID186")
    # Define a material (nominal steel in SI)
    mapdl.mp("EX", 1, 210e9)  # Elastic moduli in Pa (kg/(m*s**2))
    mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
    mapdl.mp("NUXY", 1, 0.3)  # Poisson's Ratio
    mapdl.emodif("ALL", "MAT", 1)

    # Geometry
    
    # Create a rectangular area with the hole in the middle
    center_hole_dia = 0.045 # Center hole diameter = 45 mm

    # Create the rectangle
    rect_anum = mapdl.blc4(width = 0.2, height = 0.1) # 200 mm x 100 mm

    # Create a circle in the middle of the rectangle
    circ_anum = mapdl.cyl4(0.1, 0.05, center_hole_dia/2)
    
    # Create a satellite circle at locations x = 50 mm along y = 50 mm axis
    sat_circ_anum = mapdl.cyl4(sat_holes_loc ,0.05, sat_holes_dia/2)
    sat_circ_anum2 = mapdl.cyl4(0.2-sat_holes_loc ,0.05, sat_holes_dia/2) # (200 mm - left hole location) = right hole location

    # Cut the circle out of the rectangle.
    area1= mapdl.asba(rect_anum, circ_anum)
    area2 = mapdl.asba(area1, sat_circ_anum)
    plate_with_hole_area = mapdl.asba(area2, sat_circ_anum2)
    plate_with_hole_anum=mapdl.vext(plate_with_hole_area, dz=0.003) # make volume from area (thickness = 3 mm)
    
    # Meshing

    mapdl.lesize("ALL", 0.005, layer1=1)

    mapdl.mshape(1, "3D")
    mapdl.mshkey(0)

    mapdl.vmesh('all')

    # Boundary Conditions

    mapdl.nsel('s','loc','x',0)
    mapdl.d('all','all',0)

    mapdl.nsel('S','LOC','X', 0.2)
    mapdl.nsel("R", "LOC", "Y", 0.1/2)
    single_node = mapdl.mesh.nnum[0]
    mapdl.nsel("S", "NODE", vmin=single_node, vmax=single_node)
    mapdl.f("ALL", "FZ", -100)

    mapdl.allsel()
    mapdl.finish()

    # Solve the Static Problem

    mapdl.run("/SOLU")
    mapdl.antype("STATIC")
    mapdl.solve()

    # Post-Processing

    # Grab the stress from the result
    result = mapdl.result
    nnum, stress = result.principal_nodal_stress(0)
    
    stress_left_hole = mapdl.get('sigeqv', 'node','node(sat_holes_loc,0.05+(sat_holes_dia/2),0)','s','eqv')
    stress_center_hole = mapdl.get('sigeqv', 'node','node(0.1,0.05+0.045/2,0)','s','eqv')
    
    nominal_stress_left = 6*(100)*(0.2-sat_holes_loc)/((0.1-sat_holes_dia)*(0.003**2))
    nominal_stress_center = 6*(100)*(0.1)/((0.1-0.045)*(0.003**2))
    
    KT_left = stress_left_hole / nominal_stress_left
    KT_center = stress_center_hole / nominal_stress_center
        
    return sat_holes_dia,sat_holes_loc,KT_left,KT_center

In [18]:
Sat_hole_dia=[i for i in range (4,41,2)] # diameter of satellite holes start from 4 to 40 mm with 2 mm increasing step

List=[]

for hole_dia in Sat_hole_dia:
    location = 100
    # Margin = distance from left wall to left radius of the left satellite hole
    # Margin should finally located between 0.005 to 0.076 (x = 5 mm to 76 mm) 
    # The left outer radius of center hole is locate at 77.5 mm
    # Satellite holes should located atleast out of center holes radius but not too far to the edge of plate)
    
    while location-(hole_dia/2)>76:
        location -= 1

    while location-(hole_dia/2)>=5:
        if 0 < 77.5-(location+(hole_dia/2)) < 2: # Check if the distance between satellite and center hole is less than 2 mm ?
            # If the right outer radius of the left satellite hole is too close to the center hole, skip this location
            location -= 1
        else:
            List.append([hole_dia/1000,location/1000]) # Satellite holes sizes and locations for iterations
            location -= 1

In [19]:
len(List)

1330

In [20]:
Bending=[]
i=1
for size,loc in List:
    a,b,c,d=compute_stress_con(size,loc)
    Bending.append([a,b,c,d])
    print(f'finished {i}/{len(List)} simulation ...')
    i+=1  

finished 1/1330 simulation ...
finished 2/1330 simulation ...
finished 3/1330 simulation ...
finished 4/1330 simulation ...
finished 5/1330 simulation ...
finished 6/1330 simulation ...
finished 7/1330 simulation ...
finished 8/1330 simulation ...
finished 9/1330 simulation ...
finished 10/1330 simulation ...
finished 11/1330 simulation ...
finished 12/1330 simulation ...
finished 13/1330 simulation ...
finished 14/1330 simulation ...
finished 15/1330 simulation ...
finished 16/1330 simulation ...
finished 17/1330 simulation ...
finished 18/1330 simulation ...
finished 19/1330 simulation ...
finished 20/1330 simulation ...
finished 21/1330 simulation ...
finished 22/1330 simulation ...
finished 23/1330 simulation ...
finished 24/1330 simulation ...
finished 25/1330 simulation ...
finished 26/1330 simulation ...
finished 27/1330 simulation ...
finished 28/1330 simulation ...
finished 29/1330 simulation ...
finished 30/1330 simulation ...
finished 31/1330 simulation ...
finished 32/1330 

finished 253/1330 simulation ...
finished 254/1330 simulation ...
finished 255/1330 simulation ...
finished 256/1330 simulation ...
finished 257/1330 simulation ...
finished 258/1330 simulation ...
finished 259/1330 simulation ...
finished 260/1330 simulation ...
finished 261/1330 simulation ...
finished 262/1330 simulation ...
finished 263/1330 simulation ...
finished 264/1330 simulation ...
finished 265/1330 simulation ...
finished 266/1330 simulation ...
finished 267/1330 simulation ...
finished 268/1330 simulation ...
finished 269/1330 simulation ...
finished 270/1330 simulation ...
finished 271/1330 simulation ...
finished 272/1330 simulation ...
finished 273/1330 simulation ...
finished 274/1330 simulation ...
finished 275/1330 simulation ...
finished 276/1330 simulation ...
finished 277/1330 simulation ...
finished 278/1330 simulation ...
finished 279/1330 simulation ...
finished 280/1330 simulation ...
finished 281/1330 simulation ...
finished 282/1330 simulation ...
finished 2

finished 502/1330 simulation ...
finished 503/1330 simulation ...
finished 504/1330 simulation ...
finished 505/1330 simulation ...
finished 506/1330 simulation ...
finished 507/1330 simulation ...
finished 508/1330 simulation ...
finished 509/1330 simulation ...
finished 510/1330 simulation ...
finished 511/1330 simulation ...
finished 512/1330 simulation ...
finished 513/1330 simulation ...
finished 514/1330 simulation ...
finished 515/1330 simulation ...
finished 516/1330 simulation ...
finished 517/1330 simulation ...
finished 518/1330 simulation ...
finished 519/1330 simulation ...
finished 520/1330 simulation ...
finished 521/1330 simulation ...
finished 522/1330 simulation ...
finished 523/1330 simulation ...
finished 524/1330 simulation ...
finished 525/1330 simulation ...
finished 526/1330 simulation ...
finished 527/1330 simulation ...
finished 528/1330 simulation ...
finished 529/1330 simulation ...
finished 530/1330 simulation ...
finished 531/1330 simulation ...
finished 5

finished 751/1330 simulation ...
finished 752/1330 simulation ...
finished 753/1330 simulation ...
finished 754/1330 simulation ...
finished 755/1330 simulation ...
finished 756/1330 simulation ...
finished 757/1330 simulation ...
finished 758/1330 simulation ...
finished 759/1330 simulation ...
finished 760/1330 simulation ...
finished 761/1330 simulation ...
finished 762/1330 simulation ...
finished 763/1330 simulation ...
finished 764/1330 simulation ...
finished 765/1330 simulation ...
finished 766/1330 simulation ...
finished 767/1330 simulation ...
finished 768/1330 simulation ...
finished 769/1330 simulation ...
finished 770/1330 simulation ...
finished 771/1330 simulation ...
finished 772/1330 simulation ...
finished 773/1330 simulation ...
finished 774/1330 simulation ...
finished 775/1330 simulation ...
finished 776/1330 simulation ...
finished 777/1330 simulation ...
finished 778/1330 simulation ...
finished 779/1330 simulation ...
finished 780/1330 simulation ...
finished 7

finished 1000/1330 simulation ...
finished 1001/1330 simulation ...
finished 1002/1330 simulation ...
finished 1003/1330 simulation ...
finished 1004/1330 simulation ...
finished 1005/1330 simulation ...
finished 1006/1330 simulation ...
finished 1007/1330 simulation ...
finished 1008/1330 simulation ...
finished 1009/1330 simulation ...
finished 1010/1330 simulation ...
finished 1011/1330 simulation ...
finished 1012/1330 simulation ...
finished 1013/1330 simulation ...
finished 1014/1330 simulation ...
finished 1015/1330 simulation ...
finished 1016/1330 simulation ...
finished 1017/1330 simulation ...
finished 1018/1330 simulation ...
finished 1019/1330 simulation ...
finished 1020/1330 simulation ...
finished 1021/1330 simulation ...
finished 1022/1330 simulation ...
finished 1023/1330 simulation ...
finished 1024/1330 simulation ...
finished 1025/1330 simulation ...
finished 1026/1330 simulation ...
finished 1027/1330 simulation ...
finished 1028/1330 simulation ...
finished 1029/

finished 1241/1330 simulation ...
finished 1242/1330 simulation ...
finished 1243/1330 simulation ...
finished 1244/1330 simulation ...
finished 1245/1330 simulation ...
finished 1246/1330 simulation ...
finished 1247/1330 simulation ...
finished 1248/1330 simulation ...
finished 1249/1330 simulation ...
finished 1250/1330 simulation ...
finished 1251/1330 simulation ...
finished 1252/1330 simulation ...
finished 1253/1330 simulation ...
finished 1254/1330 simulation ...
finished 1255/1330 simulation ...
finished 1256/1330 simulation ...
finished 1257/1330 simulation ...
finished 1258/1330 simulation ...
finished 1259/1330 simulation ...
finished 1260/1330 simulation ...
finished 1261/1330 simulation ...
finished 1262/1330 simulation ...
finished 1263/1330 simulation ...
finished 1264/1330 simulation ...
finished 1265/1330 simulation ...
finished 1266/1330 simulation ...
finished 1267/1330 simulation ...
finished 1268/1330 simulation ...
finished 1269/1330 simulation ...
finished 1270/

In [21]:
np.savetxt("Bending.csv", Bending,delimiter =", ", fmt ='% s')

In [36]:
for i in range(len(Bending)):
    if Bending[i][3]>1.1:
        print(Bending[i][3])

1.217668518
1.2056414452500002
1.2367096665000001
1.212640704
1.2176782777500001
1.21072328025
1.2104188965
1.2210484605
1.2148624455
1.213646577
1.2408941820000001
1.2132083205000002
1.2167742345
1.242576324
1.2437244105
1.2078272332500002
1.2335586945
1.23556460775
1.2210508365000001
1.24507009275
1.2356502675
1.2177393442500002
1.22723420325
1.24505796525
1.2429314040000001
1.2178097332500002
1.236230952
1.23462615375
1.24545573075
1.23551845725
1.2145413307500001
1.22060065875
1.2212607
1.2430524150000002
1.2188474100000002
1.2313956187500001
1.2418350945
1.24261202175
1.22130020925
1.2128946555000002
1.23857136975
1.2372808882500002
1.2157616625
1.23831136275
1.22295034125
1.2378582645
1.2403355085
1.2210417615
1.214925129
1.24052775825
1.2420529687500002
1.23148171575
1.2322819657500002
1.2169902030000002
1.2187680615
1.2178900305
1.22068142625
1.2378794835
1.23627696225
1.241079708
1.24303787025
1.24239781875
1.2362129587500001
1.2390666585
1.24247031975
1.2424014075
1.243192541